## Bag-of-Words Classifier

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
 
torch.manual_seed(1)

In [4]:
# training data set
data = [("me gusta comer en la cafeteria".split(), "SPANISH"),
        ("Give it to me".split(), "ENGLISH"),
        ("No creo que sea una buena idea".split(), "SPANISH"),
        ("No it is not a good idea to get lost at sea".split(), "ENGLISH")]

# test data set
test_data = [("Yo creo que si".split(), "SPANISH"),
             ("it is lost on me".split(), "ENGLISH")]

word_to_ix = {}
 
for sent, _ in data + test_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
print(word_to_ix)
 
VOCAB_SIZE = len(word_to_ix)
NUM_LABELS = 2
 
# classifier class we define 
class BoWClassifier(nn.Module):
    def __init__(self, num_labels, vocab_size):
        super(BoWClassifier, self).__init__()
        self.linear = nn.Linear(vocab_size, num_labels)
 
    def forward(self, bow_vec):
        return F.log_softmax(self.linear(bow_vec), dim=1)
 
# from word_to_ix to generate brow vector
def make_bow_vector(sentence, word_to_ix):
    vec = torch.zeros(len(word_to_ix))
    for word in sentence:
        ix = word_to_ix[word]
        vec[ix] += 1
    return vec.view(1, -1)
 
 
def make_target(label, label_to_ix):
    return torch.LongTensor([label_to_ix[label]])
 
 
model = BoWClassifier(NUM_LABELS, VOCAB_SIZE)
 
for param in model.parameters():
    print(param)
 
with torch.no_grad():
    sample = data[0]
    bow_vector = make_bow_vector(sample[0], word_to_ix)
    print(bow_vector)
    log_probs = model(bow_vector)
    print(log_probs)

{'me': 0, 'gusta': 1, 'comer': 2, 'en': 3, 'la': 4, 'cafeteria': 5, 'Give': 6, 'it': 7, 'to': 8, 'No': 9, 'creo': 10, 'que': 11, 'sea': 12, 'una': 13, 'buena': 14, 'idea': 15, 'is': 16, 'not': 17, 'a': 18, 'good': 19, 'get': 20, 'lost': 21, 'at': 22, 'Yo': 23, 'si': 24, 'on': 25}
Parameter containing:
tensor([[ 0.1344,  0.0406,  0.0631,  0.1465,  0.1860, -0.1301,  0.0245,  0.1464,
          0.1421,  0.1218, -0.1419, -0.1412, -0.1186,  0.0246,  0.1955, -0.1239,
          0.1045, -0.1085, -0.1844, -0.0417,  0.1130,  0.1821, -0.1218,  0.0426,
          0.1692,  0.1300],
        [ 0.1222,  0.1394,  0.1240,  0.0507, -0.1341, -0.1647, -0.0899, -0.0228,
         -0.1202,  0.0717,  0.0607, -0.0444,  0.0754,  0.0634,  0.1197,  0.1321,
         -0.0664,  0.1916, -0.0227, -0.0067, -0.1851, -0.1262, -0.1146, -0.0839,
          0.1394, -0.0641]], requires_grad=True)
Parameter containing:
tensor([-0.1466,  0.0755], requires_grad=True)
tensor([[1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 

In [0]:
label_to_ix={"SPANISH":0,"ENGLISH":1}

In [6]:
# before train, just to see a before-and-after
with torch.no_grad():
    for instance, label in test_data:
        bow_vector = make_bow_vector(instance, word_to_ix)
        log_probs = model(bow_vector)
        print(log_probs)
# before train, parameter value
print(next(model.parameters())[:, word_to_ix['creo']])
 
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)
 
# Usually, between 5 and 30 epochs is reasonable.
for epoch in range(100):
    for instance, label in data:
        # PyTorch accumulates gradients
        model.zero_grad()
        bow_vec = make_bow_vector(instance, word_to_ix)
        target = make_target(label, label_to_ix)
        log_probs = model(bow_vec)
        loss = loss_function(log_probs, target)
        loss.backward()
        optimizer.step()
with torch.no_grad():
    for instance, label in test_data:
        bow_vec = make_bow_vector(instance, word_to_ix)
        log_probs = model(bow_vec)
        print(log_probs)
print(next(model.parameters())[:, word_to_ix['creo']])

tensor([[-0.8923, -0.5271]])
tensor([[-0.4260, -1.0587]])
tensor([-0.1419,  0.0607], grad_fn=<SelectBackward>)
tensor([[-0.1481, -1.9833]])
tensor([[-2.3026, -0.1054]])
tensor([ 0.3575, -0.4387], grad_fn=<SelectBackward>)
